In [45]:
# 파일 다른 곳으로 옮기기 
# !mv 0406_word2vec_구현.ipynb kdt_ll

In [ ]:
##### 이론 #####
# word embedding

# Word2Vec : 수많은 단어를 통해 단어 벡터를 학습
#          : 유사 단어 간에 가까운 경향을 파악 

# 많은 양의 데이터 + 단순한 모델이 적은 데이터 + 복잡한 모델보다 더 낫다는 것을 가정
# 수십억 단어로 질 좋은 단어 벡터를 학습
# 유사 단어 간에는 거리가 가까운 경향이 있고, 단어는 다양한 유사도를 가짐

# Model architecture:
    # LSA보다 뛰어난 선형 정규성
    # LDA는 데이터 양이 많을수록 많은 연산을 필요로 함. (cost가 많이 듬)
    # Word2Vec 복잡도 O=E x T x Q


In [41]:
# 모델
    # CBOW (continuous bag of words) : 주변 단어를 보고 해당 타켓 단어를 예측하는 방법
        # 1. sliding window=5
        # 2. target word
        # 3. context 
        # 이렇게 3가지 정도가 있다고 볼 수 있음.
        
        # V => unique한 단어 벡터의 수
        # N => (사용자가 지정 가능한) N만큼 V의 값을 가져옴
            # N만큼의 행렬이 생성 => 여기서 평균을 낸것이 hidden vector 
            
    # skip-gram model
        # 하나의 단어를 보고 주변 단어를 예측
        # 낮은 연산 복잡도
        # 높은 정확도
        
        
# Transfer Learning / Fine Tuning
    # transfer learning : 풍부한 데이터가 있는 분야에서 훈련된 모델을 재사용
    # fine tuning : 사전 학습된 모델을 task에 적합한 데이터로 추가 학습하여 파라미터 미세 조정
    

In [ ]:
# Word2Vec 필요과정
    # Feed Forwarding
    # Backpropagation
    # Weights 갱신
    # Loss 그래프
    # 벡터간 유사도 측정
    # 가장 유사한 벡터 목록 출력
    
# CBOW 학습
    # 입력 문장 토큰화 (사용하지 않는 품사는 제거)
    # 가중치 (=파라미터=단어벡터) 초기화
    # 원핫 인코딩 (one hot encoding)
    # epoch 만큼 반복 
        # 중심단어 문맥단어 추출
        # Feed Forward
        # Loss / Gradient 계산
        # Weight Parameter 계산

## 학습 과정 ##
# 1. 학습하는 가중치가 단어 벡터
# 2. 입력 단어 (원핫 벡터)와 단어 벡터의 곱으로 hidden layer 계산 (Feed Forward 1)
# 3. Hidden layer 선택된 "하나의" 벡터 
# 4. Hidden layer와 단어 벡터의 곱으로 output layer 계산 (Feed Forward 2)
# 5. (단어간 유사도 계산) 선택된 단어 벡터와 전체 단어 벡터간 유사도 ==> softmax()
# 6. epoch 별 Loss / Gradient 계산 (W1 행렬, W2 행렬 편미분 그리고 가중치 업데이트)
# 7. CrossEntrophy (혹은 BinaryEntrophy) : weights를 gradient * learning rate 만큼 갱신


In [ ]:
##### fasttext #####
# 단어를 bag of characters로 보고 n_gram의 characters embedding 함 (skip_gram model 사용)
# Fasttext는 단어를 쪼갤 수 있음 ==> subwords (내부 단어)
    # 때문에 내부 단어를 통해 모르는 단어 (OOV)에 대해서도 다른 단어와의 유사도를 계산 가능
    
# Fasttext의 경우 단어에 n_gram을 하기 때문에 학습 경우의 수가 많아지므로 정확도가 높아지는 경향이 있음.
# Fasttext는 오타가 섞여도 일정 수준 성능을 보임 ==> 노이즈가 많은 말뭉치에 강함


***예제***

In [21]:
import numpy as np

# 입력 문장 토큰화 
sent = 'you will never know until you try'
tokens = sent.split()

# 가중치 (초기화)
V = len(tokens)
N = 4

W1 = np.random.rand(V,N)
W2 = np.random.rand(V,N)

# 원핫 인코딩 
onehot_vector = np.zeros(shape=(V,V), dtype =np.int32)
unique_tokens = set(tokens)
unique_tokens


word2idx = {}
for idx, word in enumerate(unique_tokens):
    onehot_vector[idx][idx] = 1
    word2idx[word] = idx
    
idx2word = {v:k for k,v in word2idx.items()}

# 학습 데이터 생성
# window_size = 1
window_size=1
training_data = []
for idx, token in enumerate(tokens):
    target_word = token
    context_words = []
    for window in range(1, window_size+1):
        if idx-window >= 0:
            context_words.append(tokens[idx-window])
        if idx+window < len(tokens):
            context_words.append(tokens[idx+window])  
    #context_words = [tokens[idx-1], tokens[idx+1]] # window 사이즈가 1이고 주변 단어를 가져옴
                              
    training_data.append([context_words, target_word])
    
onehot_vector

array([[1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=int32)

In [16]:
# 전체 값을 1아래로 만들어주는 함수 
import numpy as np

def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y=exp_a / sum_exp_a
    return y

a = np.array([0.3, 2.9, 4.0])
predict = softmax(a)
output = np.array([0,0,1]) 
loss = output - predict # 예측한 것과 실제 값의 차이 
loss

array([-0.01821127, -0.24519181,  0.26340309])

In [18]:
# Word2Vec 학습구현


# 4. Epoch
# - epoch 수 만큼 반복
epochs = 20

for epoch in range(epochs):
    for context_words, target_word in training_data:
        # 1. Feed Forward
        # - context 단어를 W1 행렬에 곱해서 hidden vector 만들기
        # - 만든 hidden vector들 평균내기
        vector_list = []
        for word in context_words:
            vector_list.append(onehot_vector(word2idx[word]))
        x=np.vstack(vector_list)
#         print(x)
        hidden_vector= x.dot(W1)
        hidden_vector_avg = np.sum(hidden_vector, axis=0)/len(hidden_vector)
        # - 평균 낸 hidden vector를 가지고 W2에 곱해서 Vx1 크기의 벡터로 만들기
        # - softmax 씌어숴 합이 1이 되도록 조정
#         hidden_vector_avg.reshape(1, N).dot(W2) # (1, N) x V ( 1 x V ) => V x 1
        output_layer = W2.dot(hidden_vector_avg.T) # V x N * N X (1,2) => V X (1,2)
#         print(output_layer) # (V, ) => 1차원 
        output_layer = softmax(output_layer)
        real_vector = onehot_vector[word2idx[target_word]]
        
        # 2. Loss 계산
        # - cross entrophy loss
        print(real_vector*np.log(output_layer))
        loss = -np.sum(real_vector * np.log(output_vector))
#         print(loss) # loss가 나옴. 
        loss_list.append(loss)

        # 3. Backpropagation
        # - 각 행렬에 대해서 편미분한 기울기 구하기
        # - 각 행렬에 대해 기울기 + learning rate 만큼 가중치 update

        # W2 기울기
        # Y_hat - Y
        diff = output_layer - real_vector.reshape(-1,1) # V x (1,2) - real_vector V, 1 
        print(diff) # (V, 1,2) 1,2 차원
        # Hidden Vector와 dot_product -> V x N
        # V x 1 * 1 X N => V x N
        gradient_w2 = diff.reshape(-1, 1).dot(hidden_vector.reshape(1, -1))
        print(gradient_w2)
        
        # W1 기울기 V x N                              
        # X * (W2 * diff) => 2 X V * V X N * V x (1,2) 
        # (V x N * V X (1,2)) => (1,2) X N
        # => V x (1,2) * (1,2) X N => V x N
        temp = diff.T.dot(W2) # (1,2) x V * V x N => (1,2) x N
        gradient_1 = X.T.dot(temp) # V x (1,2) * (1,2) * N => V x N
        print(gradient_w1)
        
        W1 -= learning_rate * gradient_w1
        W2 -= leanring_rate * gradient_w2
        

TypeError: 'numpy.ndarray' object is not callable

In [22]:
word2idx

{'will': 0, 'until': 1, 'know': 2, 'never': 3, 'try': 4, 'you': 5}

In [ ]:
w1v = W1[word2idx['try']] 
w2v = W1[word2idx['know']]

cos_sim = lambda x, y : x * y / (np.linalg.norm(x) * np.linalg.norm(y))
cos_sim(w1v, w2v)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(loss_list) # 반복해서 학습하다보니 튀는 경향이 있음. 학습할 데이터가 너무 적기도 함. 

In [ ]:
# Word2Vec 대량 corpus

# 1. 입력 계산 (onehot_vector를 만들지 않고, word2idx 딕션너리를 통해서 hidden_layer를 계산)
# 2. loss 계산이 전체 단어에 대해서 1, 0을 계산하는 것이 아니라 해당 target_word만 1,0 binary cross entropy 계산 
# (sigmoid 함수를 output_layer에)
# 3. (오답도 학습을 하기 위해 sampling을 통해서 0으로 예측해야 되는 단어들도 같이 학습시킨다.) W2에서 index를 뽑아서 행렬곱 
# (output_layer들 각각 나온다)

# 문맥 단어, 타겟 단어 (2천만개) 있음 ==> too many
# 4. batch_size(속도 개선) 
# 128개씩, 256개씩 

# 2천만번 계산할 것인지,
# 15만 6천번 연산할 것인지 

In [24]:
training_data2 = []
for context_words, target_word in training_data:
    for word in context_words:
        training_data2.append([word, target_word])
    
training_data2 # 위의 Word2Vec과 달리 1열로 쭉 나타냄 

[['will', 'you'],
 ['you', 'will'],
 ['never', 'will'],
 ['will', 'never'],
 ['know', 'never'],
 ['never', 'know'],
 ['until', 'know'],
 ['know', 'until'],
 ['you', 'until'],
 ['until', 'you'],
 ['try', 'you'],
 ['you', 'try']]